In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from kk.dggs import DGGS, mask_from_addresses
from kk.dggs.io import save_png, dump_text, slurp
from kk import uitools

dg = DGGS()

In [ ]:
def im_plotter(ax, dg=dg, **kwargs):
    ff = dg.mk_display_helper(**kwargs)
    
    def show(im, **kwargs):
        im, extent = ff(im.addr, im.value)
        return ax.imshow(im, extent=extent, **kwargs)
    
    return show

def build_vis_mask(aa, roi = None):
    roi = DGGS.roi_from_points(aa) if roi is None else roi
    mm = DGGS.Image(np.zeros(roi.shape, dtype='uint8'), roi)

    for a in aa:
        mm[a] = roi.scale - a.scale + 1
    
    return mm

In [ ]:
aa = slurp('mm-addresses.txt.xz', DGGS.Address)
mm = mask_from_addresses(aa)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_axes((0,0,1,1))
ax.set_axis_off()
imshow = im_plotter(ax)
imshow(mm)

for a in aa:
    if a.scale <= 9:
        uitools.plot_roi(DGGS.ROI(a), 'c-', ax=ax, alpha=0.7, linewidth=0.5)

fig.savefig('mm-grid-visualisation.png', dpi=300);

In [ ]:
palette = dict(divergent=[0xffffff,0xffffe0,0xffa474,0xdb4551,0xffffe0,0xffa474,0xdb4551,0x8b0000],
               sequential=[0xffffff, 0xffffe0,0xffd59b,0xffa474,0xf47461,0xdb4551,0xb81b34,0x8b0000])

mmc = build_vis_mask(aa, mm.roi)
mm_colour = uitools.index_to_rgb(mmc.value, palette['sequential'], alpha=0)
plt.imshow(mm_colour);

In [ ]:
save_png(f'mm-colour-{mm.addr}.png', mm_colour)